In [4]:
import MDAnalysis as mda
import matplotlib.pyplot as plt
%matplotlib notebook

In [5]:
u = mda.Universe('IRMOF.pdb')

In [6]:
data = open('irmofcharges.out').readlines()

Columns are:  
  - Type
  - Element
  - x, y, z
  - Charge

In [7]:
def nameconv(name):
    if name.startswith('Z'):
        return 'Zn'
    elif name.startswith('H'):
        return 'H'
    else:
        return name

In [8]:
types = [nameconv(l.split()[0]) for l in data]
charges = [float(l.split()[-1]) for l in data]

In [9]:
u.atoms.types = types * 8
u.atoms.names = types * 8
u.atoms.charges = charges * 8

In [10]:
set(u.atoms.types)

{'C1', 'C2', 'C3', 'H', 'O1', 'O2', 'Zn'}

In [11]:
for attype in set(u.atoms.types):
    print attype, set(u.select_atoms('type {}'.format(attype)).charges)


Zn set([1.333])
H set([0.16200000000000001])
C3 set([-0.16700000000000001])
C2 set([0.106])
C1 set([0.55800000000000005])
O2 set([-0.64100000000000001])
O1 set([-1.5640000000000001])


In [12]:
Crad = mda.topology.tables.vdwradii['C']
Orad = mda.topology.tables.vdwradii['O']

vdwrad = {}
vdwrad['C1'] = vdwrad['C2'] = vdwrad['C3'] = Crad
vdwrad['O1'] = vdwrad['O2'] = Orad
vdwrad['Zn'] = 1.42

In [13]:
u.atoms.guess_bonds(vdwradii=vdwrad)

In [14]:
u.atoms.bonds.types()

[('C3', 'H'), ('C1', 'O2'), ('C3', 'C3'), ('C2', 'C3'), ('C1', 'C2')]

In [15]:
for t in u.atoms.bonds.types():
    tg = u.bonds.select_bonds(t)
    print t, tg.values().mean(), tg.values().std()

('C3', 'H') 0.926656812337 8.16028758564e-07
('C1', 'O2') 1.30435726031 1.47018680115e-06
('C3', 'C3') 1.39000034332 7.78671818664e-07
('C2', 'C3') 1.38853360568 1.17024570397e-06
('C1', 'C2') 1.48499970138 1.26712451567e-06


In [16]:
for t in u.atoms.angles.types():
    tg = u.angles.select_bonds(t)
    print t, tg.values().mean(), tg.values().std()

('H', 'C3', 'C3') 2.0927405174 1.09467158028e-06
('O2', 'C1', 'O2') 2.20971060465 1.88309755011e-06
('C3', 'C2', 'C3') 2.09317509073 1.58557992129e-06
('C3', 'C3', 'C2') 2.0950047877 4.13288115385e-07
('C2', 'C1', 'O2') 2.03673735126 1.06346992069e-06
('C2', 'C3', 'H') 2.09543945155 1.71289948934e-06
('C1', 'C2', 'C3') 2.09500510822 8.82092773712e-07


In [19]:
u.dimensions = [51.664] * 3 + [90.0] * 3

In [20]:
u.atoms.write('withbonds.pdb', bonds='all')